In [58]:
import plotly
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from experiment import Net, InitDataset, CreateModel, LoadCheckpoint
from yaml import safe_load
import torch.nn as nn

exphash = '743a85523'

exp = 'results/%s' % exphash
with open(exp) as f:
  h = safe_load(f.read())
h = h['exp_info']  

h['dataset_split'][1] = .1
ds = InitDataset(h)


model = CreateModel(h['hyperparameters'])
LoadCheckpoint(model, 'models/%s'%exphash)

model = model.cuda()


[18:15:50] Loaded model: <All keys matched successfully>


In [59]:
val_loader = DataLoader(ds[1], batch_size=32, shuffle=False)

In [60]:
idx_to_char_map = {v:k for k,v in torch.load('../data/char_to_idx_map.pt').items()}

In [61]:
idx_to_char_map[0] = '_'

In [62]:
Torch2Py = lambda x: x.cpu().numpy().tolist()

In [76]:
def bin_acc_fn(outputs, labels):
  binary = outputs>.5
  right = binary==labels
  return right.float()

BCE_loss = nn.BCELoss(reduction='none')
def bce_loss_fn(outputs, labels):
  return BCE_loss(outputs, labels)

def Validate(val_loader, model):
  cum_acc = 0
  cum_bce_loss = 0

  inputs_ = []
  losses_ = []
  guesses_ = []
  rights_ = []
  labels_ = []
  for i, data in enumerate(val_loader):
    inputs = data['char_encoded_seqs']
    inputs_ += Torch2Py(inputs)
    labels = data['token_start_positions']
    labels_ += Torch2Py(labels)

    with torch.no_grad():
      outputs = model(inputs)
      guesses_ += Torch2Py(outputs)
      bce_loss = bce_loss_fn(outputs, labels)
      losses_ += Torch2Py(bce_loss)
      cum_bce_loss += bce_loss.mean()

      acc = bin_acc_fn(outputs, labels)
      rights_ += Torch2Py(acc)
      cum_acc += acc.mean()

  print(cum_acc/(i+1), cum_bce_loss/(i+1))
  return inputs_, losses_, rights_, labels_, guesses_

In [77]:
inputs, losses, rights, labels, guesses = Validate(val_loader, model)

tensor(0.9907, device='cuda:0') tensor(0.0380, device='cuda:0')


In [78]:
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')

In [79]:
GetChars(inputs[0])

"frequent; with only a few sequences really hitting the 'disgusting' mark. the film is much more intelligent than the usual cannibal nonsense, as it explores themes such as vietnam veterans returning into society, and turns the tables on dawn of the dead's social commentary. it analyses it's points with a sly grin on it's face, however, and it's______________________________________"

In [82]:
length = 200
for i, r in enumerate(rights):
  if sum(r)<len(r) and i%10==0:
    # underscore above the letter if a tok start was missed
    print("".join('_' if labels[i][j]==1 and x!=1 else ' ' for j, x in enumerate(rights[i][:length])))
    print("".join(str(int(guesses[i][j]*10)) if x!=1 else ' ' for j, x in enumerate(rights[i][:length])))    
    print(GetChars(inputs[i])[:length])
    print("".join(' ' if x==1 else 'X' for x in rights[i][:length]))
    print()

                                                                   _  _                                                                                                    _ _  _                       
                                                                  50 80                                                                                                    2 0  0                       
-- a 2 star rating to a film that features glamorous actors in unglamorous, unflattering roles, and is absurd in the event to boot. i don't know whats worse. putting an explanatory voice over in a fil
                                                                  XX XX                                                                                                    X X  X                       

                                                                                                                                                                                                   

                                      _                                                                    _ _                           _                                                              
                                      3                                                                    3 1                           4                                                              
baazigar which was planned after baazigar the film is a remake of nick of time, rush hour the movie does disappoint you compared to baazigar and is too cheesy and stupid in parts the all detective age
                                      X                                                                    X X                           X                                                              

                                 _ _  _                                                                      _                                                                                     


                                                                         9             0                               0                                                                                
hike to seoul, korea to see if he can get her back. not one for polite inquiry, his peeping tom act and rude rooftop razing finally yields his prize blond babe and once again she gets to enjoy an appe
                                                                         X             X                               X                                                                                

                        _  _  _      _       _ _    _  _   _  _ _  _      _   _  _       _  _   _ _ _   _     _   _  _      _  _  _ _      _  _     _  _  _ _     _       _            _  _       _     
                        25 09 09     0 9     0906 5 0 909  09 0909 0 9    09  0 909      09 0  6090909  09    09  0  09     09 09 0909  5  09 0 9   09 09 0909    09      09           09 09      